# Macro pKa calculation

This script is used for the calulation of macro pKa for SM25-46 states based on different data sources.  
First, **SM25** will be used for an example to show the process of the calculation. Also we use energy information from the file: **pKa-ECRISM-1.csv**

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data = pd.read_csv('data/pKa-ECRISM-1.csv')
print(data.columns)
data = data[['Molecule Id', 'ID tag ', 'total charge', 'pKa mean']]
data_samp = data[data['Molecule Id'] == 'SM25_micro000']
data_samp

Index(['Molecule Id', 'ID tag ', 'total charge', 'pKa mean', 'pKa SEM',
       'pKa model uncertainty', 'SMILES of extra microstate', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')


,Molecule Id,ID tag,total charge,pKa mean
0,SM25_micro000,SM25_micro003,-1,7.91
1,SM25_micro000,SM25_micro001,-1,-6.66
2,SM25_micro000,SM25_micro002,0,-7.52
3,SM25_micro000,SM25_micro004,0,-12.08
4,SM25_micro000,SM25_micro005,1,-2.33


In [3]:
list(data['Molecule Id'].unique())

['SM25_micro000',
 'SM26_micro000',
 'SM27_micro000',
 'SM28_micro000',
 'SM29_micro000',
 'SM30_micro000',
 'SM31_micro000',
 'SM32_micro000',
 'SM33_micro000',
 'SM34_micro000',
 'SM35_micro000',
 'SM36_micro000',
 'SM37_micro000',
 'SM38_micro000',
 'SM39_micro000',
 'SM40_micro000',
 'SM41_micro000',
 'SM42_micro000',
 'SM43_micro000',
 'SM44_micro000',
 'SM45_micro000',
 'SM46_micro000']

From this table, we could see we have 3 catogories of microstates with charge: -1, 0, 1, we need to calculte the energy and charge for each microstate first.

In [4]:
list(data_samp['ID tag '])

['SM25_micro003',
 'SM25_micro001',
 'SM25_micro002',
 'SM25_micro004',
 'SM25_micro005']

In [5]:
state = ['SM25_micro000']
charge = [0]
energy = [0]
for i in sorted(list(data_samp['ID tag '])):
    state.append(i)
    charge.append(int(data_samp[data['ID tag '] == i]['total charge']))
    energy.append(float(data_samp[data['ID tag '] == i]['pKa mean']))
print(state, charge, energy)

state_info = pd.DataFrame({'state_name':state, 'charge':charge, 'energy':energy})
state_info

['SM25_micro000', 'SM25_micro001', 'SM25_micro002', 'SM25_micro003', 'SM25_micro004', 'SM25_micro005'] [0, -1, 0, -1, 0, 1] [0, -6.66, -7.52, 7.91, -12.08, -2.33]


/Users/zhangyingying/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/zhangyingying/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,charge,energy,state_name
0,0,0.00,SM25_micro000
1,-1,-6.66,SM25_micro001
2,0,-7.52,SM25_micro002
3,-1,7.91,SM25_micro003
4,0,-12.08,SM25_micro004
5,1,-2.33,SM25_micro005


In [6]:
state_info.charge.unique()
state_info[state_info['charge'] == 0]

,charge,energy,state_name
0,0,0.00,SM25_micro000
2,0,-7.52,SM25_micro002
4,0,-12.08,SM25_micro004


We have 3 different states with 0 charge, so we need to calculate the probabilty of each state using [Poisson–Boltzmann equation](https://en.wikipedia.org/wiki/Poisson%E2%80%93Boltzmann_equation)

In [7]:
# Modify the unit first from kcal/mol to kt
state_info['energy'] = state_info['energy']/1.688
state_info

,charge,energy,state_name
0,0,0.000000,SM25_micro000
1,-1,-3.945498,SM25_micro001
2,0,-4.454976,SM25_micro002
3,-1,4.686019,SM25_micro003
4,0,-7.156398,SM25_micro004
5,1,-1.380332,SM25_micro005


Based on Possion Boltzmann equation, Pi = exp(-ΔE/KT)

In [8]:
# Calculate the probability of each state
state_info['Pi'] = np.exp(-state_info['energy'])
state_info

,charge,energy,state_name,Pi
0,0,0.000000,SM25_micro000,1.000000
1,-1,-3.945498,SM25_micro001,51.702060
2,0,-4.454976,SM25_micro002,86.054112
3,-1,4.686019,SM25_micro003,0.009223
4,0,-7.156398,SM25_micro004,1282.283952
5,1,-1.380332,SM25_micro005,3.976221


In [9]:
# Calculate the total prob for normalizing
state_info['Pi_total'] = state_info.groupby('charge').Pi.transform('sum')
state_info

,charge,energy,state_name,Pi,Pi_total
0,0,0.000000,SM25_micro000,1.000000,1369.338063
1,-1,-3.945498,SM25_micro001,51.702060,51.711284
2,0,-4.454976,SM25_micro002,86.054112,1369.338063
3,-1,4.686019,SM25_micro003,0.009223,51.711284
4,0,-7.156398,SM25_micro004,1282.283952,1369.338063
5,1,-1.380332,SM25_micro005,3.976221,3.976221


Transfer the prob to be normalized

In [10]:
state_info['Pi_norm'] = state_info['Pi']/state_info['Pi_total']

In [11]:
state_info

,charge,energy,state_name,Pi,Pi_total,Pi_norm
0,0,0.000000,SM25_micro000,1.000000,1369.338063,0.000730
1,-1,-3.945498,SM25_micro001,51.702060,51.711284,0.999822
2,0,-4.454976,SM25_micro002,86.054112,1369.338063,0.062844
3,-1,4.686019,SM25_micro003,0.009223,51.711284,0.000178
4,0,-7.156398,SM25_micro004,1282.283952,1369.338063,0.936426
5,1,-1.380332,SM25_micro005,3.976221,3.976221,1.000000


In [12]:
state_info['energy_norm'] = state_info['Pi_norm']*state_info['energy']
state_info

,charge,energy,state_name,Pi,Pi_total,Pi_norm,energy_norm
0,0,0.000000,SM25_micro000,1.000000,1369.338063,0.000730,0.000000
1,-1,-3.945498,SM25_micro001,51.702060,51.711284,0.999822,-3.944794
2,0,-4.454976,SM25_micro002,86.054112,1369.338063,0.062844,-0.279967
3,-1,4.686019,SM25_micro003,0.009223,51.711284,0.000178,0.000836
4,0,-7.156398,SM25_micro004,1282.283952,1369.338063,0.936426,-6.701438
5,1,-1.380332,SM25_micro005,3.976221,3.976221,1.000000,-1.380332


G = U - TS, we already get the U, now we calulate the TS using TS = ∑(Pi)ln(Pi)

In [13]:
state_info['TS'] = -state_info['Pi_norm']*np.log(state_info['Pi_norm'])
state_info

,charge,energy,state_name,Pi,Pi_total,Pi_norm,energy_norm,TS
0,0,0.000000,SM25_micro000,1.000000,1369.338063,0.000730,0.000000,0.005274
1,-1,-3.945498,SM25_micro001,51.702060,51.711284,0.999822,-3.944794,0.000178
2,0,-4.454976,SM25_micro002,86.054112,1369.338063,0.062844,-0.279967,0.173895
3,-1,4.686019,SM25_micro003,0.009223,51.711284,0.000178,0.000836,0.001540
4,0,-7.156398,SM25_micro004,1282.283952,1369.338063,0.936426,-6.701438,0.061509
5,1,-1.380332,SM25_micro005,3.976221,3.976221,1.000000,-1.380332,-0.000000


In [14]:
sum_energy = state_info.groupby('charge')[['energy_norm', 'TS']].sum().reset_index()
sum_energy

,charge,energy_norm,TS
0,-1,-3.943958,0.001718
1,0,-6.981405,0.240678
2,1,-1.380332,0.000000


In [15]:
sum_energy['G'] = sum_energy['energy_norm'] - sum_energy['TS']
sum_energy

,charge,energy_norm,TS,G
0,-1,-3.943958,0.001718,-3.945676
1,0,-6.981405,0.240678,-7.222083
2,1,-1.380332,0.000000,-1.380332


In [26]:
final_energy = pd.DataFrame(columns=['source', 'state', 'ΔG_2->1', 'ΔG_1->0', 'ΔG_1->0'])
row = ['test', 'test']
if 2 in sum_energy.charge.unique() and 1 in sum_energy.charge.unique():
    row.append(float(sum_energy[sum_energy['charge'] == 1]['G']) - float(sum_energy[sum_energy['charge'] == 2]['G']))
else:
    row.append(None)
    
if 1 in sum_energy.charge.unique() and 0 in sum_energy.charge.unique():
    row.append(float(sum_energy[sum_energy['charge'] == 0]['G']) - float(sum_energy[sum_energy['charge'] == 1]['G']))
else:
    row.append(None)

if 0 in sum_energy.charge.unique() and -1 in sum_energy.charge.unique():
    row.append(float(sum_energy[sum_energy['charge'] == -1]['G']) - float(sum_energy[sum_energy['charge'] == 0]['G']))
else:
    row.append(None)
print(row)
final_energy = pd.DataFrame(columns=['source', 'state', 'ΔG_2->1', 'ΔG_1->0', 'ΔG_1->0'])
final_energy.loc[len(final_energy)] = row
final_energy

['test', 'test', None, -5.84175098316547, 3.276406728408587]


,source,state,ΔG_2->1,ΔG_1->0,ΔG_1->0
0,test,test,None,-5.841751,3.276407


## Model for calculating all states in one file

In [29]:
def energy_calculation(data, source, final_energy):
    
    data = data[['Molecule Id', 'ID tag ', 'total charge', 'pKa mean']]
    micro_list = list(data['Molecule Id'].unique())
    for micro in micro_list:
        row = [source[:-4], micro[0:4]]
        data_samp = data[data['Molecule Id'] == micro]
        state = [micro]
        charge = [0]
        energy = [0]
        for i in sorted(list(data_samp['ID tag '])):
            state.append(i)
            charge.append(int(data_samp[data['ID tag '] == i]['total charge']))
            energy.append(float(data_samp[data['ID tag '] == i]['pKa mean']))

        state_info = pd.DataFrame({'state_name':state, 'charge':charge, 'energy':energy})
        state_info['energy'] = state_info['energy']/1.688
        state_info['Pi'] = np.exp(-state_info['energy'])
        state_info['Pi_total'] = state_info.groupby('charge').Pi.transform('sum')
        state_info['Pi_norm'] = state_info['Pi']/state_info['Pi_total']
        state_info['energy_norm'] = state_info['Pi_norm']*state_info['energy']
        state_info['TS'] = -state_info['Pi_norm']*np.log(state_info['Pi_norm'])
        sum_energy = state_info.groupby('charge')[['energy_norm', 'TS']].sum().reset_index()
        sum_energy['G'] = sum_energy['energy_norm'] - sum_energy['TS']
        sum_energy['state'] = micro[0:4]

        if 2 in sum_energy.charge.unique() and 1 in sum_energy.charge.unique():
            row.append(float(sum_energy[sum_energy['charge'] == 1]['G']) - float(sum_energy[sum_energy['charge'] == 2]['G']))
        else:
            row.append(None)

        if 1 in sum_energy.charge.unique() and 0 in sum_energy.charge.unique():
            row.append(float(sum_energy[sum_energy['charge'] == 0]['G']) - float(sum_energy[sum_energy['charge'] == 1]['G']))
        else:
            row.append(None)

        if 0 in sum_energy.charge.unique() and -1 in sum_energy.charge.unique():
            row.append(float(sum_energy[sum_energy['charge'] == -1]['G']) - float(sum_energy[sum_energy['charge'] == 0]['G']))
        else:
            row.append(None)
        final_energy.loc[len(final_energy)] = row

## Calculate for all files

In [32]:
path = './data'

files = os.listdir(path)

final_energy = pd.DataFrame(columns=['source', 'state', 'ΔG_2->1', 'ΔG_1->0', 'ΔG_0->-1'])
for f in files:
    data = pd.read_csv(path + '/' + f)
    energy_calculation(data, f, final_energy)

print(final_energy) 
final_energy.to_csv('ΔG_compare_for_diff_sources.csv')

/Users/zhangyingying/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/zhangyingying/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


                        source state  ΔG_2->1   ΔG_1->0  ΔG_0->-1
0    pKa_RodriguezPaluch_SMD_3  SM25      NaN       NaN    2.1569
1    pKa_RodriguezPaluch_SMD_3  SM26      NaN       NaN  -13.0984
2    pKa_RodriguezPaluch_SMD_3  SM27      NaN       NaN   -8.2109
3    pKa_RodriguezPaluch_SMD_3  SM28      NaN -4.312796  -11.3033
4    pKa_RodriguezPaluch_SMD_3  SM29      NaN       NaN  -8.05687
5    pKa_RodriguezPaluch_SMD_3  SM30      NaN       NaN  -7.64218
6    pKa_RodriguezPaluch_SMD_3  SM31      NaN       NaN  -7.79621
7    pKa_RodriguezPaluch_SMD_3  SM32      NaN       NaN  -8.13981
8    pKa_RodriguezPaluch_SMD_3  SM33      NaN       NaN  -7.03791
9    pKa_RodriguezPaluch_SMD_3  SM34      NaN       NaN  -8.50118
10   pKa_RodriguezPaluch_SMD_3  SM35      NaN       NaN  -7.08541
11   pKa_RodriguezPaluch_SMD_3  SM36      NaN       NaN  -4.28699
12   pKa_RodriguezPaluch_SMD_3  SM37      NaN -6.038170  -4.11766
13   pKa_RodriguezPaluch_SMD_3  SM38      NaN       NaN  -7.35782
14   pKa_R